# Analyzing comments at Hacker News

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question.

Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

We'll compare these two types of posts to determine the following:

Do Ask HN or Show HN receive more comments on average?
Do posts created at a certain time receive more comments on average?

*This dataset has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

In [1]:
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


In [2]:
headers = hn[0]
hn = hn[1:]
print(headers)
hn[:5]

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
  '8',
  '2',
  'walterbell',
  '9/30/2015 4:12']]

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print('ask_posts:',len(ask_posts))
print('show_posts:',len(show_posts))
print('other_posts:',len(other_posts))

ask_posts: 1744
show_posts: 1162
other_posts: 17194


In [4]:
total_ask_comments = 0
for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
avg_ask_comments = total_ask_comments/len(ask_posts)

print('avg ask comments:',avg_ask_comments)

avg ask comments: 14.038417431192661


In [5]:
total_show_comments = 0
for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
avg_show_comments = total_show_comments/len(show_posts)

print('avg show comments:',avg_show_comments)

avg show comments: 10.31669535283993


In average, ask posts receive more comments

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

In [6]:
import datetime as dt

In [7]:
results_list = []
for row in ask_posts:
    created_at = row[6]
    comments_n = int(row[4])
    results_list.append([created_at,comments_n])

In [8]:
print(results_list[:19])

[['8/16/2016 9:55', 6], ['11/22/2015 13:43', 29], ['5/2/2016 10:14', 1], ['8/2/2016 14:20', 3], ['10/15/2015 16:38', 17], ['9/26/2015 23:23', 1], ['4/22/2016 12:24', 4], ['11/16/2015 9:22', 1], ['2/24/2016 17:57', 1], ['6/4/2016 17:17', 2], ['9/19/2015 17:04', 7], ['9/22/2015 13:16', 1], ['6/21/2016 15:45', 1], ['1/13/2016 21:17', 4], ['10/4/2015 21:27', 4], ['1/25/2016 20:27', 2], ['10/27/2015 2:47', 3], ['1/19/2016 12:01', 1], ['3/22/2016 2:05', 22]]


Below we are going to create the counts of posts at each hour and the number of comments at each hour, in order to determine if ask posts created at a certain time are more likely to attract comments. We are focusing on ask posts because we previously determined that, on average, ask posts receive more comments than show posts

In [16]:
counts_by_hour = {}
comments_by_hour = {}
for row in results_list:
    hour_dt = dt.datetime.strptime(row[0], "%m/%d/%Y %H:%M")
    hour_string = hour_dt.strftime("%H")
    n_comments = row[1]
    
    if hour_string in counts_by_hour:
        counts_by_hour[hour_string] += 1
        comments_by_hour[hour_string] += n_comments
    else:
        counts_by_hour[hour_string] = 1
        comments_by_hour[hour_string] = n_comments

In [17]:
print(counts_by_hour)

{'19': 110, '22': 71, '09': 45, '13': 85, '07': 34, '21': 109, '15': 116, '14': 107, '04': 47, '06': 44, '12': 73, '23': 68, '18': 109, '20': 80, '00': 55, '17': 100, '08': 48, '05': 46, '16': 108, '01': 60, '03': 54, '11': 58, '10': 59, '02': 58}


In [18]:
print(comments_by_hour)

{'19': 1188, '22': 479, '09': 251, '13': 1253, '07': 267, '21': 1745, '15': 4477, '14': 1416, '04': 337, '06': 397, '12': 687, '23': 543, '18': 1439, '20': 1722, '00': 447, '17': 1146, '08': 492, '05': 464, '16': 1814, '01': 683, '03': 421, '11': 641, '10': 793, '02': 1381}


Below we are going to calculate the average of comments per hour

In [31]:
avg_by_hour = []
for hour in counts_by_hour:
    avg_by_hour.append([hour,round((comments_by_hour[hour]/counts_by_hour[hour]),2)])

In [32]:
print(avg_by_hour)

[['19', 10.8], ['22', 6.75], ['09', 5.58], ['13', 14.74], ['07', 7.85], ['21', 16.01], ['15', 38.59], ['14', 13.23], ['04', 7.17], ['06', 9.02], ['12', 9.41], ['23', 7.99], ['18', 13.2], ['20', 21.52], ['00', 8.13], ['17', 11.46], ['08', 10.25], ['05', 10.09], ['16', 16.8], ['01', 11.38], ['03', 7.8], ['11', 11.05], ['10', 13.44], ['02', 23.81]]


In order to sort the list in descending order by the average of comments per hour, we are going to first swap the columns so that the result we want to sort is on the first column

In [34]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1],row[0]])
print(swap_avg_by_hour)

[[10.8, '19'], [6.75, '22'], [5.58, '09'], [14.74, '13'], [7.85, '07'], [16.01, '21'], [38.59, '15'], [13.23, '14'], [7.17, '04'], [9.02, '06'], [9.41, '12'], [7.99, '23'], [13.2, '18'], [21.52, '20'], [8.13, '00'], [11.46, '17'], [10.25, '08'], [10.09, '05'], [16.8, '16'], [11.38, '01'], [7.8, '03'], [11.05, '11'], [13.44, '10'], [23.81, '02']]


In [38]:
sorted_swap = sorted(swap_avg_by_hour,reverse=True)
print(sorted_swap)

[[38.59, '15'], [23.81, '02'], [21.52, '20'], [16.8, '16'], [16.01, '21'], [14.74, '13'], [13.44, '10'], [13.23, '14'], [13.2, '18'], [11.46, '17'], [11.38, '01'], [11.05, '11'], [10.8, '19'], [10.25, '08'], [10.09, '05'], [9.41, '12'], [9.02, '06'], [8.13, '00'], [7.99, '23'], [7.85, '07'], [7.8, '03'], [7.17, '04'], [6.75, '22'], [5.58, '09']]


In [52]:
print('Top 5 Hours for Ask Posts Comments')

for row in sorted_swap[:5]:
    time = dt.datetime.strptime(row[1], "%H")
    time_str = dt.datetime.strftime(time, '%H:%M')
    print("{0}: {1} average comments per post".format(time_str,row[0]))

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.8 average comments per post
21:00: 16.01 average comments per post


Above there is a list of the hours in which you should create a post in order to have a higher chance of receiving comments, as these are the hours at which a higher number of comments is received. However, this is in US eastern time. Let's translate it to Central European Time

In [58]:
for row in sorted_swap[:5]:
    time = dt.datetime.strptime(row[1], "%H") + dt.timedelta(hours=6) #6 hours difference
    time_str = dt.datetime.strftime(time, '%H:%M')
    print("{0}: {1} average comments per post".format(time_str,row[0]))

21:00: 38.59 average comments per post
08:00: 23.81 average comments per post
02:00: 21.52 average comments per post
22:00: 16.8 average comments per post
03:00: 16.01 average comments per post


Above there is a list of the hours in which someone living with CET time should create a post in order to have a higher chance of receiving comments